In [31]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_columns', 100)

import warnings
warnings.filterwarnings('ignore')

In [32]:
# cargando el dataset con todas las probabilidades de compra de cada cliente
# obtenidas con el modelo de machine learning en el notebook Recomendacion_part_1.ipynb
df_probab_compra = pd.read_parquet('df_proba_compra.parquet')
df_probab_compra.head()

,prob_compra
pk_cid,
15891,0.510
16063,0.675
16203,0.715
16502,0.814
17457,0.703


Eliminando los clientes fallecidos

In [33]:
# cargando el dataset con las variables sociodemograficas de cada cliente desde S3
df_socio = pd.read_parquet("https://easy-money-project-bucket.s3.eu-west-3.amazonaws.com/sociodemographic_df.parquet")

# Uniendo el dataframe de df_full_recomend con el dataframe de socio_demographic
# que presenta solo el valor de las personas que no han fallecido
# y así poder eliminar los clientes fallecidos pasando de 456373 clientes a 456244.
# ------------------- CARGA DE SOCIO DEMOGRAPHIC -------------------
# cargando el dataframe de socio_demographic
sdg_df_bad = pd.read_parquet('https://easy-money-project-bucket.s3.eu-west-3.amazonaws.com/sociodemographic_df.parquet')
# seleccionar el valor mas actual según pk_cid, para saber si el cliente ha fallecido o no
last_partition = sdg_df_bad.groupby('pk_cid').last()
# eliminar los clientes fallecidos
last_partition = last_partition[last_partition["deceased"] != "S"]
df_socio = last_partition["deceased"]

In [34]:
# cargando el dataset con las variables de productos de cada cliente
df_prod = pd.read_parquet("https://easy-money-project-bucket.s3.eu-west-3.amazonaws.com/products_df.parquet")
df_prod

,Unnamed: 0,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,0,1375586,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
1,1,1050611,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
2,2,1050612,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
3,3,1050613,2018-01-28,1,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
4,4,1050614,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,13647304,1166765,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
5962920,13647305,1166764,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
5962921,13647306,1166763,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
5962922,13647307,1166789,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1


Agrupamos por cliente y nos quedamos con el valor maximo de cada variable, garantizando que si en algun periodo el cliente usó un producto pues se sobre entiende que tiene contratado dicho producto.

In [35]:
df_prod = df_prod.groupby("pk_cid").agg("max").reset_index()
df_prod

,pk_cid,Unnamed: 0,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,15891,5319232,2018-08-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
1,16063,13026461,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
2,16203,13026405,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
3,16502,13026070,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,1
4,17457,13026626,2019-05-28,0,0,0,0,0,1,0,0,0.000,0.000,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456368,1553685,13336818,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
456369,1553686,13336817,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
456370,1553687,13336816,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
456371,1553688,13336815,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0


In [36]:
# se hace un merge de los dos datasets anteriores para tener un dataset completo con todas las variables
df_full = pd.merge(df_prod, df_socio, on=["pk_cid"], how="inner")
df_full

,pk_cid,Unnamed: 0,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,deceased
0,15891,5319232,2018-08-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,N
1,16063,13026461,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,N
2,16203,13026405,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,N
3,16502,13026070,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,1,N
4,17457,13026626,2019-05-28,0,0,0,0,0,1,0,0,0.000,0.000,0,0,1,0,1,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456239,1553685,13336818,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,N
456240,1553686,13336817,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,N
456241,1553687,13336816,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,N
456242,1553688,13336815,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,N


Ahora nos quedamos con las columnas que nos interesan para el analisis

In [37]:
columnas_relev = ['pk_cid','short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 
                  'long_term_deposit',"em_account_pp",	"credit_card"	,"payroll"	,"pension_plan",
                  "payroll_account"	,"emc_account",	"debit_card",	"em_account_p"	,"em_acount"]

df_full = df_full[columnas_relev]
df_full

,pk_cid,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,15891,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
1,16063,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
2,16203,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
3,16502,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,1
4,17457,0,0,0,0,0,1,0,0,0.000,0.000,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456239,1553685,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
456240,1553686,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
456241,1553687,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
456242,1553688,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0


# Enfoque Global

A continuacion se crea un ranking de productos para recomendar, basándonos en los que el cliente no tiene y ordenándolos según la probabilidad de compra. Esta estrategia permite mejorar la eficiencia de la campaña de marketing y aumentar la satisfacción y fidelización de los clientes.


In [38]:
# 1. Identificar las columnas de productos (todas excepto 'pk_cid')
product_cols = df_full.columns.drop('pk_cid')

# 2. Sumar los valores de cada columna de productos
product_totals = df_full[product_cols].sum()

# 3. Crear un DataFrame con los totales
product_ranking = product_totals.reset_index()
product_ranking.columns = ['Producto', 'Total']

# 4. Ordenar los productos por total en orden descendente
product_ranking = product_ranking.sort_values(by='Total', ascending=False)
product_ranking

,Producto,Total
14,em_acount,332641.000
12,debit_card,63494.000
10,payroll_account,31557.000
9,pension_plan,28178.000
11,emc_account,27598.000
8,payroll,26884.000
5,long_term_deposit,9079.000
7,credit_card,8520.000
0,short_term_deposit,5319.000
4,securities,2083.000


In [39]:
product_ranking["ranking"] = product_ranking["Total"].rank(ascending=False)
product_ranking

,Producto,Total,ranking
14,em_acount,332641.000,1.000
12,debit_card,63494.000,2.000
10,payroll_account,31557.000,3.000
9,pension_plan,28178.000,4.000
11,emc_account,27598.000,5.000
8,payroll,26884.000,6.000
5,long_term_deposit,9079.000,7.000
7,credit_card,8520.000,8.000
0,short_term_deposit,5319.000,9.000
4,securities,2083.000,10.000


Se crea un DataFrame que contiene para cada cliente (pk_cid) una lista de recomendaciones de productos basado en el ranking. Esto se realiza con la funcion get_top_n_recommendations, que identifica el producto de mayor ranking que el cliente no tiene y se sugiere como recomendación. A continuación se describe especificamente que hace la función:

1. Identifica los productos que el cliente no tiene.
2. Se crea un nuevo ranking de esos productos (que el cliente no tiene), basado en el Ranking principal
3. Selecciona los productos primeros en el ranking y se retornar en una lista de 3 parametros(dichos prametros se pueden cambiar y poner directamente n=1 y te devuelve un solo producto)

In [40]:

def get_top_n_recommendations(row, n=3):
    # Obtener los productos que el cliente tiene
    client_products = row[product_cols]
     # Identificar los productos que el cliente NO tiene
    not_owned_products = client_products[client_products == 0].index.tolist()
    # Filtrar el ranking para incluir solo los productos que el cliente no tiene
    ranking = product_ranking.set_index('Producto')['ranking']
    not_owned_ranking = ranking.loc[not_owned_products]
     # Verificar si hay productos que recomendar
    if not not_owned_ranking.empty:
        # Obtener los top N productos con el ranking más alto
        recommended_products = not_owned_ranking.nsmallest(n).index.tolist()
        return recommended_products
    else:
        return None  # El cliente tiene todos los productos

# Aplicar la función para obtener las top N recomendaciones
df_full['recomendaciones'] = df_full.apply(get_top_n_recommendations, axis=1, n=3)

# Crear el DataFrame final con las recomendaciones
df_recommendations = df_full[['pk_cid', 'recomendaciones']]

# Mostrar el DataFrame de recomendaciones
print(df_recommendations)

         pk_cid                               recomendaciones
0         15891   [debit_card, payroll_account, pension_plan]
1         16063      [em_acount, debit_card, payroll_account]
2         16203   [debit_card, payroll_account, pension_plan]
3         16502   [debit_card, payroll_account, pension_plan]
4         17457  [payroll_account, pension_plan, emc_account]
...         ...                                           ...
456239  1553685      [em_acount, debit_card, payroll_account]
456240  1553686      [em_acount, debit_card, payroll_account]
456241  1553687      [em_acount, debit_card, payroll_account]
456242  1553688      [em_acount, debit_card, payroll_account]
456243  1553689      [em_acount, debit_card, payroll_account]

[456244 rows x 2 columns]


Para quedarnos con la primera recomendación

In [41]:
df_recommendations["recomend"] = df_recommendations["recomendaciones"].apply(lambda x: x[0] if x else None)
df_recommendations.head()

,pk_cid,recomendaciones,recomend
0,15891,"[debit_card, payroll_account, pension_plan]",debit_card
1,16063,"[em_acount, debit_card, payroll_account]",em_acount
2,16203,"[debit_card, payroll_account, pension_plan]",debit_card
3,16502,"[debit_card, payroll_account, pension_plan]",debit_card
4,17457,"[payroll_account, pension_plan, emc_account]",payroll_account


Ahora se agrega el precio de cada producto

In [42]:
# Definir las listas de productos
cuenta_bancaria_products = ['em_account_pp', 'payroll', 'payroll_account', 'emc_account', 'debit_card', 'em_account_p', 'em_acount']
inversion_products = ['short_term_deposit', 'funds', 'securities', 'long_term_deposit', 'pension_plan']
financiacion_products = ['loans', 'mortgage', 'credit_card']

In [43]:
products_precios = {
    'em_account_pp': 10, 'payroll': 10, 'payroll_account': 10, 'emc_account': 10, 'debit_card': 10, 'em_account_p': 10, 'em_acount': 10,
    'short_term_deposit': 40, 'funds': 40, 'securities': 40, 'long_term_deposit': 40, 'pension_plan': 40,
    'loans': 60, 'mortgage': 60, 'credit_card': 60
}

df_recommendations["precio"] = df_recommendations["recomend"].apply(lambda x: products_precios[x] if x else None)
df_recommendations

,pk_cid,recomendaciones,recomend,precio
0,15891,"[debit_card, payroll_account, pension_plan]",debit_card,10
1,16063,"[em_acount, debit_card, payroll_account]",em_acount,10
2,16203,"[debit_card, payroll_account, pension_plan]",debit_card,10
3,16502,"[debit_card, payroll_account, pension_plan]",debit_card,10
4,17457,"[payroll_account, pension_plan, emc_account]",payroll_account,10
...,...,...,...,...
456239,1553685,"[em_acount, debit_card, payroll_account]",em_acount,10
456240,1553686,"[em_acount, debit_card, payroll_account]",em_acount,10
456241,1553687,"[em_acount, debit_card, payroll_account]",em_acount,10
456242,1553688,"[em_acount, debit_card, payroll_account]",em_acount,10


In [44]:
df_recommendations.value_counts("recomend")

recomend
debit_card            278179
em_acount             123603
payroll_account        41694
emc_account             9353
long_term_deposit       1714
pension_plan            1474
credit_card               90
payroll                   76
short_term_deposit        55
securities                 6
Name: count, dtype: int64

In [45]:
df_recommendations["precio"].value_counts()

precio
10    452905
40      3249
60        90
Name: count, dtype: int64

Análisis de los Resultados:

- Concentración de Recomendaciones: La gran mayoría de las recomendaciones se concentran en los primeros tres productos del ranking global, que a su vez representan al grupo de productos Cuentas.

- Poca Diversidad en Recomendaciones: Los demás productos reciben muy pocas recomendaciones en comparación.

- Posible Falta de Relevancia Personalizada: Este enfoque global no considera las diferencias individuales entre los clientes.

Limitaciones del Enfoque Global:

- Falta de Personalización: Todos los clientes reciben recomendaciones basadas en el mismo ranking, sin considerar sus características individuales. Es probable que las necesidades y preferencias varíen significativamente entre diferentes grupos de clientes.
  
- Recomendaciones Poco Relevantes: Algunos clientes pueden no estar interesados en los productos más recomendados. Productos que podrían ser más relevantes para ciertos clientes están siendo ignorados debido al ranking global.

	
- Oportunidades Perdidas: Al no segmentar, se podría estar perdiendo oportunidades para ofrecer productos que tienen mayor potencial de aceptación en ciertos grupos.



---

Teniendo en cuenta todo lo anterior se toma la decision de hacer un analisis de ranking de productos por grupos, a partir de la segmentación obtenida en la Tarea 2:

- Se adaptan las recomendaciones según las características y necesidades específicas de cada grupo lo que aumenta la probabilidad de aceptación.

- Mayor y mejor diversidad en recomendaciones ya que productos que son menos recomendados en el enfoque global podrían ser más relevantes para ciertos grupos y, por tanto, recomendados con mayor frecuencia.
  
Por lo tanto, implementar un análisis de recomendaciones por grupos de clientes mejorará significativamente la relevancia de las recomendaciones y se aprovechará mejor el potencial del catálogo de productos.

---

# Enfoque por grupos de clientes (por cluster)

Cargando el dataset de la segmentación de la Tarea 2

In [46]:
df_clustering = pd.read_parquet("s3://easy-money-project-bucket/df_clustering.parquet")
df_clustering

,num_products_contracts,entry_date,active_customer,mes_partition,age,cuentas,ahorro_inversion,financiacion,entry_channel_group_Canal Secundario,salary_category_Ingreso Bajo,salary_category_Ingreso Medio,cluster_4,cluster_5,cluster_6
pk_cid,,,,,,,,,,,,,,
15891,0.000,0.486,0.500,3.000,2.429,-0.773,0.000,0.000,1.000,0.000,0.000,0,4,5
16063,-1.000,0.641,0.714,0.000,2.643,-1.545,0.000,0.000,1.000,0.000,0.000,0,4,5
16203,0.000,0.688,0.833,0.000,3.214,-0.258,0.000,0.000,1.000,0.000,0.000,0,4,5
16502,1.000,0.573,0.889,0.000,2.357,0.687,0.000,0.000,1.000,0.000,0.000,0,4,5
17457,2.000,0.058,1.000,0.000,2.071,1.455,0.000,0.074,1.000,0.000,1.000,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1553685,-1.000,0.904,0.000,0.000,1.929,-1.545,0.000,0.000,1.000,0.000,0.000,1,1,2
1553686,-1.000,0.904,0.000,0.000,0.357,-1.545,0.000,0.000,1.000,0.000,1.000,1,1,2
1553687,-1.000,0.904,0.000,0.000,-0.286,-1.545,0.000,0.000,1.000,0.000,0.000,1,1,2


In [47]:
# Uniendo el DataFrame de recomendaciones con el DataFrame de clustering
df_full = pd.merge(df_full, df_clustering["cluster_6"], on="pk_cid", how="left")
df_full.drop("recomendaciones", axis=1, inplace=True)

In [48]:
df_full

,pk_cid,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,cluster_6
0,15891,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,5
1,16063,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,5
2,16203,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,5
3,16502,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,1,5
4,17457,0,0,0,0,0,1,0,0,0.000,0.000,0,0,1,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456239,1553685,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,2
456240,1553686,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,2
456241,1553687,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,2
456242,1553688,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,2


In [49]:
df_full["cluster_6"].value_counts()

cluster_6
0    121000
3    120365
2    101683
5     70079
4     36084
1      7033
Name: count, dtype: int64

Diviediendo el dataset en varios dataset por los grupos existentes

In [50]:
df_cluster_0 = df_full[df_full["cluster_6"] == 0]
df_cluster_1 = df_full[df_full["cluster_6"] == 1]
df_cluster_2 = df_full[df_full["cluster_6"] == 2]
df_cluster_3 = df_full[df_full["cluster_6"] == 3]
df_cluster_4 = df_full[df_full["cluster_6"] == 4]
df_cluster_5 = df_full[df_full["cluster_6"] == 5]


In [51]:
df_cluster_0

,pk_cid,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,cluster_6
271,84263,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0
594,151847,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,0,0
908,211353,0,0,0,0,0,0,0,0,0.000,0.000,0,0,1,0,1,0
1035,238428,0,0,0,1,0,0,0,0,0.000,0.000,0,0,0,0,1,0
1556,362642,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442286,1535916,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0
442292,1535923,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0
442293,1535924,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0
442294,1535925,0,0,0,0,0,0,0,0,0.000,0.000,0,0,1,0,1,0


In [52]:
# 1. Identificar las columnas de productos (todas excepto 'pk_cid')
product_cols = ['short_term_deposit', 'loans', 'mortgage', 'funds', 'securities',
                'long_term_deposit', 'em_account_pp', 'credit_card', 'payroll',
       'pension_plan', 'payroll_account', 'emc_account', 'debit_card',
       'em_account_p', 'em_acount']

dicc_cluster = {"cluster_0": df_cluster_0, "cluster_1": df_cluster_1,
                 "cluster_2": df_cluster_2, "cluster_3": df_cluster_3, 
                 "cluster_4": df_cluster_4, "cluster_5": df_cluster_5}

df_rankind_cluster = pd.DataFrame()

for i,j in dicc_cluster.items():
       # 2. Sumar los valores de cada columna de productos
       product_totals = j[product_cols].sum()
       
       # 3. Crear un DataFrame con los totales
       product_ranking = product_totals.reset_index()
       product_ranking.columns = [f'Producto_{i}', f'Total_{i}']
       
       # 4. Ordenar los productos por total en orden descendente
       product_ranking.sort_values(by=f'Total_{i}', ascending=False, inplace=True, ignore_index=True)
       
       df_rankind_cluster = pd.concat([df_rankind_cluster, product_ranking], axis=1)
       #df_rankind_cluster.reset_index(drop=True, inplace=True)

df_rankind_cluster

,Producto_cluster_0,Total_cluster_0,Producto_cluster_1,Total_cluster_1,Producto_cluster_2,Total_cluster_2,Producto_cluster_3,Total_cluster_3,Producto_cluster_4,Total_cluster_4,Producto_cluster_5,Total_cluster_5
0,em_acount,119952.000,em_acount,2244.000,em_acount,3362.000,em_acount,119933.000,payroll_account,27216.000,em_acount,61158.000
1,debit_card,9729.000,debit_card,465.000,credit_card,270.000,debit_card,8735.000,pension_plan,26898.000,debit_card,18469.000
2,emc_account,2084.000,emc_account,258.000,emc_account,202.000,emc_account,1030.000,payroll,26223.000,emc_account,11393.000
3,payroll_account,985.000,short_term_deposit,233.000,debit_card,172.000,payroll_account,913.000,em_acount,25992.000,long_term_deposit,4681.000
4,long_term_deposit,227.000,payroll_account,141.000,short_term_deposit,159.000,short_term_deposit,282.000,debit_card,25924.000,short_term_deposit,3799.000
5,credit_card,156.000,pension_plan,113.000,payroll_account,41.000,long_term_deposit,140.000,emc_account,12631.000,payroll_account,2261.000
6,pension_plan,95.000,payroll,102.000,pension_plan,23.000,pension_plan,126.000,credit_card,7001.000,credit_card,1003.000
7,securities,82.000,long_term_deposit,89.000,long_term_deposit,9.000,securities,69.000,long_term_deposit,3933.000,pension_plan,923.000
8,funds,43.000,credit_card,33.000,funds,8.000,payroll,62.000,securities,1355.000,securities,564.000
9,payroll,39.000,funds,5.000,securities,8.000,credit_card,57.000,funds,1122.000,funds,459.000


Se puede observar que el analisis del ranking de los productos por clusters varía, siendo más especifico con respecto al enfoque general.

Los productos más populares varían entre clusters. Por ejemplo:

- payroll_account es el más popular en Cluster 4 pero no lo es en los demás.
- credit_card es el segundo más popular en Cluster 2, pero no tiene la misma posición en otros clusters.

Los datos muestran claramente que los clientes en diferentes clusters tienen preferencias y comportamientos distintos. Esto fundamenta la necesidad de hacer el análisis por grupos.

## Recomendación para cada cluster

Agrupando los ranking por cluster

In [53]:
ranking_cluster_0 = df_rankind_cluster[["Producto_cluster_0", "Total_cluster_0"]]
ranking_cluster_1 = df_rankind_cluster[["Producto_cluster_1", "Total_cluster_1"]]
ranking_cluster_2 = df_rankind_cluster[["Producto_cluster_2", "Total_cluster_2"]]
ranking_cluster_3 = df_rankind_cluster[["Producto_cluster_3", "Total_cluster_3"]]
ranking_cluster_4 = df_rankind_cluster[["Producto_cluster_4", "Total_cluster_4"]]
ranking_cluster_5 = df_rankind_cluster[["Producto_cluster_5", "Total_cluster_5"]]

ranking_cluster_0["ranking"] = ranking_cluster_0["Total_cluster_0"].rank(ascending=False)
ranking_cluster_1["ranking"] = ranking_cluster_1["Total_cluster_1"].rank(ascending=False)
ranking_cluster_2["ranking"] = ranking_cluster_2["Total_cluster_2"].rank(ascending=False)
ranking_cluster_3["ranking"] = ranking_cluster_3["Total_cluster_3"].rank(ascending=False)
ranking_cluster_4["ranking"] = ranking_cluster_4["Total_cluster_4"].rank(ascending=False)
ranking_cluster_5["ranking"] = ranking_cluster_5["Total_cluster_5"].rank(ascending=False)


Utilizando la misma funcion que para el enfoque general, se obtiene la recomendación de cada producto para cada cliente por cluster.

In [54]:
dicc_cluster_ranking = {0:ranking_cluster_0, 1:ranking_cluster_1,
                2:ranking_cluster_2, 3:ranking_cluster_3,
                4:ranking_cluster_4, 5:ranking_cluster_5}


for i,j in dicc_cluster_ranking.items():
    
    def get_top_n_recommendations(row, n=3):
        # Obtener los productos que el cliente tiene
        client_products = row[product_cols]
        # Identificar los productos que el cliente NO tiene
        not_owned_products = client_products[client_products == 0].index.tolist()
        # Filtrar el ranking para incluir solo los productos que el cliente no tiene
        ranking = j.set_index(f'Producto_cluster_{i}')['ranking']
        not_owned_ranking = ranking.loc[not_owned_products]
        # Verificar si hay productos que recomendar
        if not not_owned_ranking.empty:
            # Obtener los top N productos con el ranking más alto
            recommended_products = not_owned_ranking.nsmallest(n).index.tolist()
            return recommended_products
        else:
            return None
        
    df_cluster = dicc_cluster[f"cluster_{i}"]
    # Aplicar la función para obtener las top N recomendaciones
    df_cluster[f'recomendaciones_{i}'] = df_cluster.apply(get_top_n_recommendations, axis=1, n=1)
    # Crear el DataFrame final con las recomendaciones
    if i == 0:
        df_cluster_recomend_0 = df_cluster[['pk_cid', f'recomendaciones_{i}']]
        print(df_cluster_recomend_0[f"recomendaciones_{i}"].value_counts())
        print(df_cluster_recomend_0.head())
    elif i == 1:
        df_cluster_recomend_1 = df_cluster[['pk_cid', f'recomendaciones_{i}']]
        print  (df_cluster_recomend_1[f"recomendaciones_{i}"].value_counts())
        print(df_cluster_recomend_1.head())
    elif i == 2:
        df_cluster_recomend_2 = df_cluster[['pk_cid', f'recomendaciones_{i}']]
        print(df_cluster_recomend_2[f"recomendaciones_{i}"].value_counts())
        print(df_cluster_recomend_2.head())
    elif i == 3:
        df_cluster_recomend_3 = df_cluster[['pk_cid', f'recomendaciones_{i}']]
        print(df_cluster_recomend_3[f"recomendaciones_{i}"].value_counts())
        print(df_cluster_recomend_3.head())
    elif i == 4:
        df_cluster_recomend_4 = df_cluster[['pk_cid', f'recomendaciones_{i}']]
        print(df_cluster_recomend_4[f"recomendaciones_{i}"].value_counts())
        print(df_cluster_recomend_4.head())
    elif i == 5:
        df_cluster_recomend_5 = df_cluster[['pk_cid', f'recomendaciones_{i}']]
        print(df_cluster_recomend_5[f"recomendaciones_{i}"].value_counts())
        print(df_cluster_recomend_5.head())




recomendaciones_0
[debit_card]           110304
[emc_account]            9610
[em_acount]              1048
[payroll_account]          35
[long_term_deposit]         3
Name: count, dtype: int64
      pk_cid recomendaciones_0
271    84263      [debit_card]
594   151847       [em_acount]
908   211353     [emc_account]
1035  238428      [debit_card]
1556  362642      [debit_card]
recomendaciones_1
[em_acount]             4789
[debit_card]            1854
[emc_account]            365
[short_term_deposit]      24
[payroll_account]          1
Name: count, dtype: int64
     pk_cid recomendaciones_1
43    28470       [em_acount]
72    35063      [debit_card]
152   53660      [debit_card]
175   61348       [em_acount]
349  103082       [em_acount]
recomendaciones_2
[em_acount]      98321
[credit_card]     3361
[emc_account]        1
Name: count, dtype: int64
    pk_cid recomendaciones_2
15   19812       [em_acount]
17   20303       [em_acount]
25   22491       [em_acount]
35   25482       [em_a

Como se puede observar, por cada grupo de cliente se obtiene la recomendación de producto para cada cliente.

## Incluyendo las probabilidades, precio y beneficio en cada cluster

Mergeando los dataset de recomendacion por cada grupo con su probabilidad de compra

Se agrega para cada cluster el precio de compra del producto recomendado, el beneficio (prprob_compra * precio) y por ultimo el cluster a que pertenece.

In [55]:
df_cluster_recomend_0 = df_cluster_recomend_0.merge(df_probab_compra, on="pk_cid", how="left")
df_cluster_recomend_0["precio"] = df_cluster_recomend_0["recomendaciones_0"].apply(lambda x: products_precios[x[0]] if x else None)
df_cluster_recomend_0["beneficio"] = df_cluster_recomend_0["prob_compra"] * df_cluster_recomend_0["precio"]
df_cluster_recomend_0["cluster"] = 0
df_cluster_recomend_0

,pk_cid,recomendaciones_0,prob_compra,precio,beneficio,cluster
0,84263,[debit_card],0.762,10,7.619,0
1,151847,[em_acount],0.729,10,7.292,0
2,211353,[emc_account],0.835,10,8.347,0
3,238428,[debit_card],0.972,10,9.722,0
4,362642,[debit_card],0.732,10,7.321,0
...,...,...,...,...,...,...
120995,1535916,[debit_card],0.549,10,5.485,0
120996,1535923,[debit_card],0.547,10,5.467,0
120997,1535924,[debit_card],0.502,10,5.025,0
120998,1535925,[emc_account],0.591,10,5.909,0


In [56]:
df_cluster_recomend_1 = df_cluster_recomend_1.merge(df_probab_compra, on="pk_cid", how="left")
df_cluster_recomend_1["precio"] = df_cluster_recomend_1["recomendaciones_1"].apply(lambda x: products_precios[x[0]] if x else None)
df_cluster_recomend_1["beneficio"] = df_cluster_recomend_1["prob_compra"] * df_cluster_recomend_1["precio"]
df_cluster_recomend_1["cluster"] = 1
df_cluster_recomend_1

,pk_cid,recomendaciones_1,prob_compra,precio,beneficio,cluster
0,28470,[em_acount],0.225,10,2.252,1
1,35063,[debit_card],0.939,10,9.391,1
2,53660,[debit_card],0.488,10,4.876,1
3,61348,[em_acount],0.225,10,2.252,1
4,103082,[em_acount],0.206,10,2.059,1
...,...,...,...,...,...,...
7028,1520003,[em_acount],0.208,10,2.079,1
7029,1520017,[em_acount],0.177,10,1.771,1
7030,1520058,[em_acount],0.129,10,1.289,1
7031,1520126,[em_acount],0.179,10,1.787,1


In [57]:
df_cluster_recomend_2 = df_cluster_recomend_2.merge(df_probab_compra, on="pk_cid", how="left")
df_cluster_recomend_2["precio"] = df_cluster_recomend_2["recomendaciones_2"].apply(lambda x: products_precios[x[0]] if x else None)
df_cluster_recomend_2["beneficio"] = df_cluster_recomend_2["prob_compra"] * df_cluster_recomend_2["precio"]
df_cluster_recomend_2["cluster"] = 2
df_cluster_recomend_2

,pk_cid,recomendaciones_2,prob_compra,precio,beneficio,cluster
0,19812,[em_acount],0.657,10,6.566,2
1,20303,[em_acount],0.739,10,7.386,2
2,22491,[em_acount],0.746,10,7.458,2
3,25482,[em_acount],0.685,10,6.850,2
4,30078,[em_acount],0.703,10,7.026,2
...,...,...,...,...,...,...
101678,1553685,[em_acount],0.092,10,0.917,2
101679,1553686,[em_acount],0.165,10,1.653,2
101680,1553687,[em_acount],0.216,10,2.159,2
101681,1553688,[em_acount],0.240,10,2.403,2


In [58]:
df_cluster_recomend_3 = df_cluster_recomend_3.merge(df_probab_compra, on="pk_cid", how="left")
df_cluster_recomend_3["precio"] = df_cluster_recomend_3["recomendaciones_3"].apply(lambda x: products_precios[x[0]] if x else None)
df_cluster_recomend_3["beneficio"] = df_cluster_recomend_3["prob_compra"] * df_cluster_recomend_3["precio"]
df_cluster_recomend_3["cluster"] = 3
df_cluster_recomend_3

,pk_cid,recomendaciones_3,prob_compra,precio,beneficio,cluster
0,169963,[debit_card],0.611,10,6.107,3
1,341953,[debit_card],0.863,10,8.629,3
2,387424,[debit_card],0.805,10,8.051,3
3,454675,[debit_card],0.561,10,5.614,3
4,509285,[debit_card],0.651,10,6.505,3
...,...,...,...,...,...,...
120360,1553296,[debit_card],0.192,10,1.917,3
120361,1553303,[debit_card],0.201,10,2.011,3
120362,1553393,[debit_card],0.088,10,0.877,3
120363,1553429,[debit_card],0.216,10,2.159,3


In [59]:
df_cluster_recomend_4 = df_cluster_recomend_4.merge(df_probab_compra, on="pk_cid", how="left")
df_cluster_recomend_4["precio"] = df_cluster_recomend_4["recomendaciones_4"].apply(lambda x: products_precios[x[0]] if x else None)
df_cluster_recomend_4["beneficio"] = df_cluster_recomend_4["prob_compra"] * df_cluster_recomend_4["precio"]
df_cluster_recomend_4["cluster"] = 4
df_cluster_recomend_4

,pk_cid,recomendaciones_4,prob_compra,precio,beneficio,cluster
0,17457,[payroll_account],0.703,10,7.026,4
1,17970,[em_acount],0.726,10,7.256,4
2,20333,[payroll_account],0.891,10,8.908,4
3,26018,[payroll_account],0.943,10,9.427,4
4,26163,[payroll_account],0.651,10,6.513,4
...,...,...,...,...,...,...
36079,1551597,[payroll_account],0.260,10,2.605,4
36080,1551712,[payroll_account],0.208,10,2.079,4
36081,1551832,[payroll_account],0.260,10,2.605,4
36082,1552055,[payroll_account],0.184,10,1.835,4


In [60]:
df_cluster_recomend_5 = df_cluster_recomend_5.merge(df_probab_compra, on="pk_cid", how="left")
df_cluster_recomend_5["precio"] = df_cluster_recomend_5["recomendaciones_5"].apply(lambda x: products_precios[x[0]] if x else None)
df_cluster_recomend_5["beneficio"] = df_cluster_recomend_5["prob_compra"] * df_cluster_recomend_5["precio"]
df_cluster_recomend_5["cluster"] = 5
df_cluster_recomend_5


,pk_cid,recomendaciones_5,prob_compra,precio,beneficio,cluster
0,15891,[debit_card],0.510,10,5.101,5
1,16063,[em_acount],0.675,10,6.752,5
2,16203,[debit_card],0.715,10,7.149,5
3,16502,[debit_card],0.814,10,8.137,5
4,17590,[em_acount],0.764,10,7.645,5
...,...,...,...,...,...,...
70074,1553626,[em_acount],0.165,10,1.652,5
70075,1553637,[em_acount],0.260,10,2.605,5
70076,1553644,[em_acount],0.196,10,1.958,5
70077,1553662,[em_acount],0.180,10,1.797,5


Se renombra la columna recomendaciones para evitar problemas y se unen en un mismo dataframe los df anteriores.

In [61]:
df_cluster_recomend_0.rename(columns={"recomendaciones_0":"recomendacion"}, inplace=True)
df_cluster_recomend_1.rename(columns={"recomendaciones_1":"recomendacion"}, inplace=True)
df_cluster_recomend_2.rename(columns={"recomendaciones_2":"recomendacion"}, inplace=True)
df_cluster_recomend_3.rename(columns={"recomendaciones_3":"recomendacion"}, inplace=True)
df_cluster_recomend_4.rename(columns={"recomendaciones_4":"recomendacion"}, inplace=True)
df_cluster_recomend_5.rename(columns={"recomendaciones_5":"recomendacion"}, inplace=True)

# quiero unir los 6 dataframes anteriores en uno solo, pero uno debajo del otro
df_recommendations_final = pd.concat([
    df_cluster_recomend_0,
    df_cluster_recomend_1,
    df_cluster_recomend_2,
    df_cluster_recomend_3,
    df_cluster_recomend_4,
    df_cluster_recomend_5
], ignore_index=True)
df_recommendations_final

,pk_cid,recomendacion,prob_compra,precio,beneficio,cluster
0,84263,[debit_card],0.762,10,7.619,0
1,151847,[em_acount],0.729,10,7.292,0
2,211353,[emc_account],0.835,10,8.347,0
3,238428,[debit_card],0.972,10,9.722,0
4,362642,[debit_card],0.732,10,7.321,0
...,...,...,...,...,...,...
456239,1553626,[em_acount],0.165,10,1.652,5
456240,1553637,[em_acount],0.260,10,2.605,5
456241,1553644,[em_acount],0.196,10,1.958,5
456242,1553662,[em_acount],0.180,10,1.797,5


## Análisis de los 10mil clientes

En este analisis se van a tener en cuenta 3 escenarios:

- Clientes que ofrecen mayor beneficio
- Clientes con mayor probabilidades de compra
- Analisis mixto de clientes.

Y partiremos recordando la siguiente tabla:

In [93]:
# Datos de la primera tabla
data1 = {
    'Cluster': [0, 1, 2, 3, 4, 5],
    'Nombre': [
        "0_Jóvenes de Baja Actividad",
        "1_Nuevos Clientes de Bajo Compromiso",
        "2_Jóvenes Inactivos Sin Productos",
        "3_Jóvenes Activos en Crecimiento",
        "4_Clientes Premium Altamente Comprometidos",
        "5_Seniors Comprometidos con Potencial"
        
    ],
    'Número de Productos Contratados': ['Moderado', 'Bajo', 'Muy Bajo', 'Moderado', 'Alto', 'Moderado'],
    'Actividad del Cliente': ['Media Baja', 'Baja', 'Muy Baja', 'Media', 'Alta', 'Media Alta'],
    'Ingreso': ['Ingreso Bajo/Medio', 'Ingreso Medio Alto', 'Ingreso Medio Bajo', 'Ingreso Medio', 'Mixto/Medio Alto', 'Medio Alto'],
    'Canal de Entrada': [
        'Muy Bajo uso de secundarios',
        'Alto uso de secundarios',
        'Bajo uso de secundarios',
        'Bajo uso en secundarios',
        'Medio uso en secundarios',
        'Alta uso en secundarios'
    ],
    'Edad': ['Jóvenes', 'Mediana Edad', 'Jóvenes', 'Jóvenes', 'Mediana Edad', 'Mayores'],
    'Uso de Cuentas, Ahorro e Inversión': [
        'Bajo', 'Bajo', 'Muy Bajo', 'Medio Bajo', 'Alto', 'Moderado'
    ],
    'Fecha de Ingreso': [
        'Marzo 2016', 'Marzo 2018', 'Agosto 2017',
        'Abril 2018', 'Febrero 2017', 'Octubre 2017'],
    
    "Antiguedad": ["Casi 3 años", "Un año", "Casi de 2 años", "Un año", "Más de 2 años", "Casi 2 años"],
    'Descripción Breve':  [
        # Clúster 0
        'Jóvenes con baja actividad y pocos productos contratados.',
        # Clúster 1
        'Clientes recientes de mediana edad con bajo compromiso y actividad.',
        # Clúster 2
        'Jóvenes inactivos sin productos, en riesgo de abandono.',
        # Clúster 3
        'Jóvenes muy activos con potencial de crecimiento financiero.',
        # Clúster 4
        'Clientes altamente comprometidos con múltiples productos y servicios complejos.',
        # Clúster 5
        'Clientes mayores activos con potencial para productos financieros más complejos.'
    ],
    'Estrategias Clave': [
        # Clúster 0
        '- Programa de recompensas por uso activo para incentivar actividad.',
        # Clúster 1
        '- Ofertas personalizadas para aumentar contratación.\n- Mejora de canales digitales.',
        # Clúster 2
        '- Reactivación con incentivos.\n- Encuestas para identificar barreras.',
        # Clúster 3
        '- Educación financiera.\n- Promociones adaptadas a jóvenes.',
        # Clúster 4
        '- Programas de fidelización exclusivos.\n- Asesoría financiera personalizada.',
        # Clúster 5
        '- Ofrecer productos financieros más complejos.\n- Eventos para fortalecer la relación.'
    ]
}

# Datos de la segunda tabla
data2 = {
    'Cluster': [0, 1, 2, 3, 4, 5],
    'Número Medio de Productos': [1.093, 0.485, 0.039, 1.080, 3.871, 1.401],
    'Clientes Activos (%)': [28.4, 16.6, 3.7, 40.6, 94.5, 75.4],
    'Meses como Cliente': [4.996, 10.551, 4.950, 4.989, 4.991, 4.962],
    'Edad Media': [25.868, 35.202, 28.888, 23.277, 37.405, 49.626],
    'Uso de Cuentas (%)': [14.2, 3.8, 0.1, 13.9, 19.1, 13.8],
    'Ahorro e Inversión (%)': [0.0, 0.3, 0.0, 0.0, 14.7, 0.6],
    'Financiación (%)': [1.0, 0.8, 0.0, 0.8, 14.8, 4.2],
    'Canal Secundario (%)': [7.9, 71.6, 27.3, 11.6, 49.2, 70.8],
}

# Crear DataFrames a partir de los diccionarios
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Unir los DataFrames en base a la columna 'Cluster'
df = pd.merge(df1, df2, on= 'Cluster')

# Reorganizar las columnas para mejorar la legibilidad
df = df[[
    'Nombre', 'Edad', 'Edad Media', "Fecha de Ingreso", "Antiguedad",
    'Número de Productos Contratados', 'Número Medio de Productos',
    'Actividad del Cliente', 'Clientes Activos (%)',
    'Canal de Entrada', 'Canal Secundario (%)',
    'Uso de Cuentas, Ahorro e Inversión', 'Uso de Cuentas (%)',
    'Ahorro e Inversión (%)', 'Financiación (%)',
    'Descripción Breve', 'Estrategias Clave'
]]

# Mostrar la tabla resumen
df

,Nombre,Edad,Edad Media,Fecha de Ingreso,Antiguedad,Número de Productos Contratados,Número Medio de Productos,Actividad del Cliente,Clientes Activos (%),Canal de Entrada,Canal Secundario (%),"Uso de Cuentas, Ahorro e Inversión",Uso de Cuentas (%),Ahorro e Inversión (%),Financiación (%),Descripción Breve,Estrategias Clave
0,0_Jóvenes de Baja Actividad,Jóvenes,25.868,Marzo 2016,Casi 3 años,Moderado,1.093,Media Baja,28.400,Muy Bajo uso de secundarios,7.900,Bajo,14.200,0.000,1.000,Jóvenes con baja actividad y pocos productos c...,- Programa de recompensas por uso activo para ...
1,1_Nuevos Clientes de Bajo Compromiso,Mediana Edad,35.202,Marzo 2018,Un año,Bajo,0.485,Baja,16.600,Alto uso de secundarios,71.600,Bajo,3.800,0.300,0.800,Clientes recientes de mediana edad con bajo co...,- Ofertas personalizadas para aumentar contrat...
2,2_Jóvenes Inactivos Sin Productos,Jóvenes,28.888,Agosto 2017,Casi de 2 años,Muy Bajo,0.039,Muy Baja,3.700,Bajo uso de secundarios,27.300,Muy Bajo,0.100,0.000,0.000,"Jóvenes inactivos sin productos, en riesgo de ...",- Reactivación con incentivos.\n- Encuestas pa...
3,3_Jóvenes Activos en Crecimiento,Jóvenes,23.277,Abril 2018,Un año,Moderado,1.080,Media,40.600,Bajo uso en secundarios,11.600,Medio Bajo,13.900,0.000,0.800,Jóvenes muy activos con potencial de crecimien...,- Educación financiera.\n- Promociones adaptad...
4,4_Clientes Premium Altamente Comprometidos,Mediana Edad,37.405,Febrero 2017,Más de 2 años,Alto,3.871,Alta,94.500,Medio uso en secundarios,49.200,Alto,19.100,14.700,14.800,Clientes altamente comprometidos con múltiples...,- Programas de fidelización exclusivos.\n- Ase...
5,5_Seniors Comprometidos con Potencial,Mayores,49.626,Octubre 2017,Casi 2 años,Moderado,1.401,Media Alta,75.400,Alta uso en secundarios,70.800,Moderado,13.800,0.600,4.200,Clientes mayores activos con potencial para pr...,- Ofrecer productos financieros más complejos....


### Clientes que ofrecen mayor beneficio

En este escenario se ordenan los clientes que brindan mayor beneficio para la empresa y se seleccionan los primeros 10mil

In [63]:
df_recommendations_final = df_recommendations_final.sort_values(by='beneficio', ascending=False)
df_seleccionados = df_recommendations_final.head(10000)

In [64]:
# distribucion de recomendaciones por cluster
df_seleccionados["cluster"].value_counts()

cluster
4    4150
2    3358
5    1912
0     517
1      37
3      26
Name: count, dtype: int64

In [66]:
df_seleccionados["recomendacion"].value_counts()

recomendacion
[credit_card]           4722
[debit_card]            1702
[pension_plan]          1013
[payroll_account]        899
[long_term_deposit]      823
[em_acount]              564
[emc_account]            111
[short_term_deposit]      59
[securities]              47
[payroll]                 47
[funds]                   13
Name: count, dtype: int64

In [67]:
# suma de los beneficios de las recomendaciones
df_seleccionados.sum()

pk_cid                                                 12414376545
recomendacion    [credit_card, credit_card, credit_card, credit...
prob_compra                                               8132.630
precio                                                      394750
beneficio                                               296917.995
cluster                                                      32991
dtype: object

In [68]:
# ordenar los clientes por probabilidad de compra
df_seleccionados.sort_values(by='prob_compra', ascending=True)

,pk_cid,recomendacion,prob_compra,precio,beneficio,cluster
172109,1290265,[credit_card],0.165,60,9.909,2
218816,1517336,[credit_card],0.167,60,10.019,2
217745,1514323,[credit_card],0.167,60,10.019,2
218272,1516008,[credit_card],0.185,60,11.077,2
207104,1479517,[credit_card],0.187,60,11.240,2
...,...,...,...,...,...,...
395757,1100936,[debit_card],0.994,10,9.940,5
395267,1092543,[em_acount],0.994,10,9.943,5
350881,917273,[payroll_account],0.995,10,9.947,4
398183,1123415,[debit_card],0.995,10,9.952,5


Como resultado de este escenario, se puede observar que se presenta una mejor distribución de productos segun el comportamiento de los cluster.

Además, el beneficio alcanzado por esta campaña sería como mínimo de 296917.995 euros y como máximo de 394750 euros.

Por otra parte, se considera que este escenario no es el más adecuado ya que tiende a maximizar el beneficio y no tiene mucho en cuenta las probalidades de compra, es decir, presenta clientes con probabilidades de compra muy bajas pero como tiene recomendado un producto con alto precio pues la ganancia es mayor con respecto a otros.

---

### Clientes con mayor probabilidades de compra

En este escenario se realiza un analisis similar al anterior pero a partir de las probabilidades de compra, ordenando los clientes que presentan mayor probalidad de compra y nos quedamos con los 10mil primeros.

In [69]:
los_10000 = df_recommendations_final.sort_values(by='prob_compra', ascending=False).head(10000)
los_10000.sum()

pk_cid                                                 12101384266
recomendacion    [debit_card, debit_card, payroll_account, em_a...
prob_compra                                               9613.097
precio                                                      108360
beneficio                                               104216.227
cluster                                                      26571
dtype: object

In [70]:
los_10000["cluster"].value_counts()

cluster
0    3212
3    2385
5    2210
4    2009
2     146
1      38
Name: count, dtype: int64

In [71]:
los_10000["recomendacion"].value_counts()

recomendacion
[debit_card]            7063
[payroll_account]       1129
[em_acount]              929
[emc_account]            604
[credit_card]             88
[pension_plan]            67
[payroll]                 55
[long_term_deposit]       40
[securities]              13
[funds]                    7
[short_term_deposit]       5
Name: count, dtype: int64

En este escenario se puede observar un poco de desbalance a la hora de recomendar los productos principalmente debit card y lo mismo sucede a la hora de distribuir productos por cluster, ademas el beneficio que se puede lograr alcanzar es bajo siendo como minimo 104216.227 y como maximo 108360. A pesar de que seria un escenerio bastante realista por las probalidades de compra altas, no esta garantizando un benificio adecuado.


---

### Analisis mixto de clientes. 

En este escenario se analiza el comportamiento de la probablidad de compra por cada cluster estableciendo los umbrales de probabilidad de compra adecuados para cada grupo. A continuación, analizaremos los datos de cada cluster y definiremos los umbrales basándonos en la distribución de las probabilidades de compra (prob_compra).

Para cluster_0:

In [72]:
df_recommendations_final[df_recommendations_final["cluster"] ==0].describe()

,pk_cid,prob_compra,precio,beneficio,cluster
count,121000.000,121000.000,121000.000,121000.000,121000.000
mean,1124677.003,0.832,10.001,8.325,0.000
std,69112.793,0.120,0.149,1.203,0.000
min,84263.000,0.039,10.000,0.388,0.000
25%,1065315.750,0.783,10.000,7.827,0.000
50%,1120657.000,0.875,10.000,8.754,0.000
75%,1184149.250,0.904,10.000,9.038,0.000
max,1535926.000,0.991,40.000,36.101,0.000


- La media y mediana son altas, indicando que la mayoría de los clientes tienen una probabilidad de compra elevada.
- El 75% de los clientes tiene una probabilidad de compra superior a 0.783.
- La distribución está sesgada hacia probabilidades altas, con baja dispersión.

Propuesta de Umbral:

- Umbral de Probabilidad de Compra: > 0.80
  
- Al establecer un umbral de 0.80, nos enfocamos en aproximadamente el 65% de los clientes (entre el percentil 25% y el máximo), quienes tienen una alta probabilidad de compra. Esto maximiza la eficiencia al dirigir recursos hacia los clientes más propensos a comprar.

Para cluster 1:

In [73]:
df_recommendations_final[df_recommendations_final["cluster"] ==1].describe()

,pk_cid,prob_compra,precio,beneficio,cluster
count,7033.000,7033.000,7033.000,7033.000,7033.000
mean,1377090.854,0.429,10.102,4.357,1.000
std,165597.932,0.281,1.750,3.130,0.000
min,28470.000,0.005,10.000,0.045,1.000
25%,1354071.000,0.175,10.000,1.753,1.000
50%,1433374.000,0.411,10.000,4.109,1.000
75%,1476963.000,0.683,10.000,6.861,1.000
max,1520374.000,0.988,40.000,38.561,1.000


- La media y mediana son moderadas, indicando una distribución amplia y dispersa.
- El 25% superior de los clientes tiene una probabilidad de compra superior a 0.683.
- Existe un segmento con alta probabilidad de compra, aunque es menor en comparación con el total del clúster.

Propuesta de Umbral:

- Umbral de Probabilidad de Compra: > 0.70

- Estableciendo un umbral de 0.70, nos enfocamos en el 25% superior de clientes con mayor probabilidad de compra. Dado que son clientes de alto potencial, es aceptable incluir a más clientes potenciales para fomentar su compromiso y aprovechar oportunidades de crecimiento.

In [74]:
df_recommendations_final[df_recommendations_final["cluster"] ==2].describe()

,pk_cid,prob_compra,precio,beneficio,cluster
count,101683.000,101683.000,101683.000,101683.000,101683.000
mean,1307402.922,0.577,11.653,6.961,2.000
std,178995.396,0.224,8.939,7.307,0.000
min,19812.000,0.005,10.000,0.046,2.000
25%,1187074.500,0.385,10.000,3.851,2.000
50%,1335566.000,0.627,10.000,6.331,2.000
75%,1470095.500,0.747,10.000,7.573,2.000
max,1553689.000,0.965,60.000,57.920,2.000


- La mediana es 0.627, indicando que el 50% de los clientes tiene una probabilidad de compra superior a este valor.
- Hay una dispersión moderada, con un segmento significativo de clientes con probabilidad de compra entre 0.60 y 0.75.

Propuesta de Umbral:

- Umbral de Probabilidad de Compra: > 0.60

- Al enfocar en clientes con probabilidad de compra superior al 60%, abarcamos al 50% superior. Esto permite dirigir esfuerzos hacia clientes que, aunque inactivos, muestran mayor predisposición a reactivar su relación con la entidad.

Para cluster 3

In [75]:
df_recommendations_final[df_recommendations_final["cluster"] ==3].describe()

,pk_cid,prob_compra,precio,beneficio,cluster
count,120365.000,120365.000,120365.000,120365.000,120365.000
mean,1390958.634,0.782,10.001,7.824,3.000
std,81296.844,0.169,0.173,1.693,0.000
min,169963.000,0.013,10.000,0.125,3.000
25%,1316870.000,0.723,10.000,7.230,3.000
50%,1409219.000,0.852,10.000,8.519,3.000
75%,1459649.000,0.890,10.000,8.896,3.000
max,1553571.000,0.980,40.000,36.044,3.000


- La media y mediana son altas, similar al Clúster 0.
- El 50% de los clientes tiene una probabilidad de compra superior a 0.852.

Propuesta de Umbral:

- Umbral de Probabilidad de Compra: > 0.80
  
- Estableciendo un umbral de 0.80, nos enfocamos en el 50% superior con mayor probabilidad de compra. Esto maximiza el retorno al dirigir estrategias hacia los clientes más propensos a adquirir nuevos productos y con mayor potencial de crecimiento.

Para cluster 4

In [76]:
df_recommendations_final[df_recommendations_final["cluster"] ==4].describe()

,pk_cid,prob_compra,precio,beneficio,cluster
count,36084.000,36084.000,36084.000,36084.000,36084.000
mean,1227788.579,0.736,13.148,9.715,4.000
std,188837.539,0.144,11.063,8.629,0.000
min,17457.000,0.031,10.000,0.313,4.000
25%,1114483.250,0.645,10.000,6.497,4.000
50%,1239933.500,0.727,10.000,7.410,4.000
75%,1372631.000,0.867,10.000,8.948,4.000
max,1552364.000,0.995,60.000,59.212,4.000


- La mediana es 0.727, con un 25% superior de clientes con probabilidad de compra mayor a 0.867.
- Dado su alto valor y compromiso, es estratégico abarcar una mayor proporción de este clúster.

Propuesta de Umbral:

- Umbral de Probabilidad de Compra: > 0.75.

- Al establecer el umbral en 0.75, nos enfocamos en los clientes con mayor predisposición a comprar (aproximadamente el 40% superior), maximizando oportunidades de venta cruzada y fidelización.

In [77]:
df_recommendations_final[df_recommendations_final["cluster"] ==5].describe()

,pk_cid,prob_compra,precio,beneficio,cluster
count,70079.000,70079.000,70079.000,70079.000,70079.000
mean,1302275.948,0.695,10.180,7.084,5.000
std,234062.338,0.145,2.316,2.267,0.000
min,15891.000,0.022,10.000,0.222,5.000
25%,1199440.000,0.630,10.000,6.297,5.000
50%,1367079.000,0.694,10.000,6.952,5.000
75%,1477078.000,0.771,10.000,7.715,5.000
max,1553684.000,0.996,40.000,39.548,5.000


- La media y mediana están cerca de 0.70, indicando una distribución centrada.
- El 25% superior tiene una probabilidad de compra mayor a 0.771.

Propuesta de Umbral:

- Umbral de Probabilidad de Compra: > 0.70

- Estableciendo el umbral en 0.70, abarcamos al 50% superior de clientes con mayor probabilidad de compra. Esto permite dirigir estrategias hacia seniors con mayor predisposición, aprovechando su potencial para productos financieros más complejos.

In [98]:
# tabla resumen de umbrales por cluster
umbrales = { "Jóvenes de Baja Actividad": f"> 0.80",
             "Nuevos Clientes de Bajo Compromiso": f"> 0.70",
             "Jóvenes Inactivos Sin Productos": f"> 0.60",
             "Jóvenes Activos en Crecimiento": f"> 0.80",
             "Clientes Premium Altamente Comprometidos": f"> 0.75",
             "Seniors Comprometidos con Potencial": f"> 0.70"
           }

umbrales_df = pd.DataFrame(umbrales.items(), columns=["Cluster", "Umbral de Probabilidad de Compra"])
umbrales_df


,Cluster,Umbral de Probabilidad de Compra
0,Jóvenes de Baja Actividad,> 0.80
1,Nuevos Clientes de Bajo Compromiso,> 0.70
2,Jóvenes Inactivos Sin Productos,> 0.60
3,Jóvenes Activos en Crecimiento,> 0.80
4,Clientes Premium Altamente Comprometidos,> 0.75
5,Seniors Comprometidos con Potencial,> 0.70


Aplicaremos un filtrado por umbrales de cluster al df_recommendations_final

In [99]:
umbrales_por_cluster = {
    0: 0.80,
    1: 0.70,
    2: 0.60,
    3: 0.80,
    4: 0.75,
    5: 0.70
}

clusters_filtrados = []

# Filtrar los clientes por cluster y umbral de probabilidad de compra en df_recommendations_final
for i, df_cluster in df_recommendations_final.groupby("cluster"):
    umbral = umbrales_por_cluster.get(i)
    df_cluster_filtrado = df_cluster[df_cluster['prob_compra'] >= umbral]
    clusters_filtrados.append(df_cluster_filtrado)

In [100]:
# llevando clusters filtrados a un solo dataframe
df_clusters_filtrados = pd.concat(clusters_filtrados, ignore_index=True)
df_clusters_filtrados

,pk_cid,recomendacion,prob_compra,precio,beneficio,cluster
0,1086587,[long_term_deposit],0.903,40,36.101,0
1,1132045,[debit_card],0.991,10,9.906,0
2,1071796,[debit_card],0.991,10,9.906,0
3,1019255,[debit_card],0.991,10,9.906,0
4,1123868,[emc_account],0.988,10,9.879,0
...,...,...,...,...,...,...
276192,1443387,[debit_card],0.700,10,7.001,5
276193,1456343,[debit_card],0.700,10,7.001,5
276194,1448261,[debit_card],0.700,10,7.001,5
276195,1444176,[debit_card],0.700,10,7.001,5


In [101]:
# comprobando comportamiento de los clusters filtrados segun la probabilidad de compra
# es decir, si se cumple el umbral de probabilidad de compra, el minimo umbral es de 0.60
df_clusters_filtrados.describe()

,pk_cid,prob_compra,precio,beneficio,cluster
count,276197.000,276197.000,276197.000,276197.000,276197.000
mean,1257559.831,0.848,10.688,9.038,2.131
std,169842.918,0.081,5.660,4.685,1.703
min,16203.000,0.600,10.000,6.000,0.000
25%,1127997.000,0.821,10.000,8.231,0.000
50%,1262065.000,0.871,10.000,8.717,2.000
75%,1403707.000,0.903,10.000,9.038,3.000
max,1548217.000,0.996,60.000,59.212,5.000


A continuacion, se ordena por beneficio esperado y seleccionamos los 10mil clientes

In [102]:
df_todos_clusters = df_clusters_filtrados.sort_values(by='beneficio', ascending=False)
df_seleccionados = df_todos_clusters.head(10000)
distribucion_clusters_final = df_seleccionados['cluster'].value_counts()
print("Distribución final de clientes por cluster:")
print(distribucion_clusters_final)

Distribución final de clientes por cluster:
cluster
4    2803
2    2696
5    2045
0    1657
3     770
1      29
Name: count, dtype: int64


- Clúster 4 (Clientes Premium Altamente Comprometidos): 2803 clientes. Este es el clúster más grande, lo cual tiene sentido, ya que se trata de clientes de alto valor y compromiso, probablemente el grupo más atractivo para la empresa.

- Clúster 2 (Jóvenes Inactivos Sin Productos): 2696 clientes. Este es el segundo clúster más grande, lo que sugiere que tienes una cantidad significativa de clientes jóvenes que no están activos, y que podrían requerir estrategias de reactivación o retención.

- Clúster 5 (Seniors Comprometidos con Potencial): 2045 clientes. Este tamaño es razonable, indicando una buena base de clientes mayores con potencial para productos financieros más complejos.

- Clúster 0 (Jóvenes de Baja Actividad): 1657 clientes. Este clúster más pequeño refleja a los clientes jóvenes que no están muy activos pero tienen cierto potencial de crecimiento. Deberían ser objetivo de campañas de activación.

- Clúster 3 (Jóvenes Activos en Crecimiento): 770 clientes. Este grupo, aunque pequeño, representa a los clientes jóvenes con mayor probabilidad de crecimiento, por lo que es un clúster valioso a pesar de su tamaño.

- Clúster 1 (Nuevos Clientes de Bajo Compromiso): 29 clientes. Esta es una cifra baja pero es coherente dado su bajo potencial de compra.

In [105]:
df_seleccionados["recomendacion"].value_counts()

recomendacion
[debit_card]            3756
[credit_card]           3281
[payroll_account]       1026
[em_acount]              721
[pension_plan]           402
[long_term_deposit]      402
[emc_account]            303
[payroll]                 49
[securities]              28
[short_term_deposit]      22
[funds]                   10
Name: count, dtype: int64

- debit_card es el producto más recomendado, con 3756 clientes (37.56% del total).
- credit_card es el segundo más recomendado, con 3281 clientes (32.81% del total).
- Los productos de mayor precio, como credit_card, long_term_deposit, pension_plan, están bien representados.
- Los productos de menor precio, como debit_card, em_acount, emc_account, también tienen una alta presencia, lo que refleja un buen equilibrio.

In [104]:
df_seleccionados.sum()

pk_cid                                                 12359860645
recomendacion    [credit_card, credit_card, credit_card, credit...
prob_compra                                               9065.260
precio                                                      289970
beneficio                                               245677.015
cluster                                                      29168
dtype: object

- El beneficio total esperado de la campaña es de 245677.015, basado en las probabilidades de compra y los precios de los productos, con un máximo de 289970.
  


In [107]:
df_seleccionados[df_seleccionados["cluster"] == 0]["recomendacion"].value_counts()    

recomendacion
[debit_card]           1566
[emc_account]            74
[em_acount]              16
[long_term_deposit]       1
Name: count, dtype: int64

- Recomendación Principal: [debit_card]
- Clientes con baja contratación de productos y actividad moderada, lo que hace que la recomendación de productos básicos como la tarjeta de débito sea coherente.
- Estrategia Adecuada: Promover productos de bajo compromiso como cuentas básicas y tarjetas de débito para incrementar su actividad y fidelización.

In [108]:
df_seleccionados[df_seleccionados["cluster"] == 1]["recomendacion"].value_counts() 

recomendacion
[debit_card]            11
[short_term_deposit]     6
[emc_account]            6
[em_acount]              6
Name: count, dtype: int64

- Recomendaciones Principales: [debit_card], [short_term_deposit], [emc_account], [em_acount]
- Pocos clientes en este clúster con una distribución variada de recomendaciones. Predomina la recomendación de productos básicos como tarjetas y cuentas.
- Estrategia Adecuada: Incentivar el uso de productos financieros básicos con ofertas o promociones para mejorar su compromiso.


In [110]:
df_seleccionados[df_seleccionados["cluster"] == 2]["recomendacion"].value_counts() 

recomendacion
[credit_card]    2647
[em_acount]        49
Name: count, dtype: int64

- Recomendación Principal: [credit_card]
- La gran mayoría de clientes de este clúster recibió recomendaciones de tarjetas de crédito, lo que sugiere que podrían estar listos para explorar productos financieros más avanzados.
- Estrategia Adecuada: Fomentar el uso de productos de financiación como las tarjetas de crédito para activar su potencial de gasto.

In [88]:
df_seleccionados[df_seleccionados["cluster"] == 3]["recomendacion"].value_counts() 

recomendacion
[debit_card]            721
[emc_account]            43
[short_term_deposit]      2
[long_term_deposit]       1
[em_acount]               1
Name: count, dtype: int64

- Recomendación Principal: [debit_card]
- Similar al clúster 0, los clientes de este grupo están recibiendo recomendaciones de productos básicos, pero con mayor énfasis en tarjetas de débito, dado su perfil de crecimiento.
- Estrategia Adecuada: Promocionar productos fáciles de usar como tarjetas de débito para aumentar la participación de este grupo joven y en crecimiento.

In [111]:
df_seleccionados[df_seleccionados["cluster"] == 4]["recomendacion"].value_counts() 

recomendacion
[payroll_account]      1024
[credit_card]           634
[pension_plan]          402
[em_acount]             345
[long_term_deposit]     193
[emc_account]            72
[payroll]                49
[debit_card]             46
[securities]             28
[funds]                  10
Name: count, dtype: int64

- Recomendaciones Diversas: [payroll_account], [credit_card], [pension_plan], [em_acount], [long_term_deposit]
- Este clúster incluye clientes premium altamente comprometidos. Las recomendaciones reflejan productos avanzados como cuentas nómina, tarjetas de crédito y planes de pensión.
- Estrategia Adecuada: Ofrecer productos más complejos y de mayor valor, como inversiones a largo plazo y planes de pensión, para mantener su lealtad y aumentar su valor.

In [112]:
df_seleccionados[df_seleccionados["cluster"] == 5]["recomendacion"].value_counts() 

recomendacion
[debit_card]            1410
[em_acount]              304
[long_term_deposit]      208
[emc_account]            106
[short_term_deposit]      15
[payroll_account]          2
Name: count, dtype: int64

- Recomendación Principal: [debit_card]
- Predomina la recomendación de tarjetas de débito, con algunos productos más avanzados como depósitos a largo plazo. Esto indica que son seniors con potencial pero con uso básico de productos.
- Estrategia Adecuada: Promover productos más avanzados como depósitos a largo plazo o cuentas especializadas, complementando las tarjetas de débito.

Conclusión

- Este escenario presenta mejor comportamiento que los anteriores.
  
- La seleccion de los clientes ha sido basada en datos, aplicando umbrales de probabilidad adecuados basados en el análisis estadístico de cada cluster.

- La distribución de clientes entre los clusters es coherente y equilibrada.
- Las recomendaciones de productos se alinean con las características y necesidades de cada cluster.




Recomendaciones:

- Es fundamental monitorear y ajustar los umbrales periódicamente en función de cambios en el comportamiento de los clientes y nuevos datos disponibles para mantener la efectividad de las estrategias.

In [113]:
# Guardar el DataFrame de recomendaciones seleccionadas para los 10000 clientes
#df_seleccionados.to_parquet("df_seleccionados.parquet")

---

#### Tasa de respuesta y Retorno sobre la Inversión

Teniendo en cuenta que:

- Tasa de Respuesta: Proporción de clientes a quienes se les envía una campaña y que efectivamente responden (adquieren el producto recomendado).

- ROI (Retorno sobre la Inversión): Medida de la rentabilidad de una inversión. Se calcula como:
	
ROI = Beneficio Neto / Costo de la Inversión * 100

Donde:

Beneficio Neto = Ingresos Totales - Costo de la Inversión


ESTIMACIÓN DE LA TASA DE RESPUESTA

Dado que prob_compra ya representa la probabilidad individual de que cada cliente compre, la tasa de respuesta estimada se calcula sumando todas las probabilidades y dividiéndolo por el número total de clientes seleccionados.


Tasa de Respuesta Estimada =  SUM (prob_compra)/ Número de Clientes Seleccionados * 100


- La suma de prob_compra es 9,065.26, por tanto la tasa de respuesta estimada es de:

9,065.26 / 10,000 * 100 = 90.65%



Aunque parezca que la tasa de respuesta del 90% es muy alta, principalmente para el sector financiero, es este caso puede ser razonable:

- Selección óptima de clientes: Se ha seleccionado a los 10,000 clientes con mayor probabilidad de compra, según las predicciones del modelo. Estos clientes están en la cima del ranking en cuanto a probabilidad de compra, por lo que se podría esperar que una gran parte de ellos responda positivamente.
- Modelo altamente preciso: Con una precision del 79.2% y un recall del 93.9%, el modelo implementado es eficaz para identificar a los clientes que comprarán, y cuando predice que un cliente va a comprar, es correcto el 79% de las veces.
- Segmentación focalizada y personalizada: La campaña está personalizada y dirigida a los clientes correctos, con mensajes y productos que resuenen con ellos, la tasa de respuesta puede ser muy alta.


ESTIMACIÓN DEL ROI

In [117]:
# 1. Calcular el beneficio total esperado
beneficio_total_esperado = df_seleccionados['beneficio'].sum()

# 2. Definir y calcular todos los costos de la campaña
# aqui se definen algunos costos de la campaña estimados
costo_por_correo = 0.50
costo_diseño = 500
costo_plataforma = 1000
costo_incentivos = 2000

costo_total = (10000 * costo_por_correo) + costo_diseño + costo_plataforma + costo_incentivos

# 3. Definir una tasa de respuesta realista basada en datos históricos o benchmarks
tasa_respuesta_real = 0.90  

# 4. Calcular el beneficio esperado basado en la tasa de respuesta real
beneficio_respuestas_real = beneficio_total_esperado * tasa_respuesta_real

# 5. Calcular el beneficio neto
beneficio_neto_real = beneficio_respuestas_real - costo_total

# 6. Calcular el ROI realista
roi_real = (beneficio_neto_real / costo_total) * 100

# Mostrar los resultados
print(f"Beneficio Total Esperado: ${beneficio_total_esperado:,.2f}")
print(f"Beneficio Esperado de las Respuestas: ${beneficio_respuestas_real:,.2f}")
print(f"Costo Total de la Campaña: ${costo_total:,.2f}")
print(f"Beneficio Neto Real: ${beneficio_neto_real:,.2f}")
print(f"ROI: {roi_real:.2f}%")

Beneficio Total Esperado: $245,677.02
Beneficio Esperado de las Respuestas: $221,109.31
Costo Total de la Campaña: $8,500.00
Beneficio Neto Real: $212,609.31
ROI: 2501.29%


Un ROI del 2501.29% indica que por cada dólar invertido, se espera recuperar $25.01 en beneficio neto.

Es importante destacar que tanto el cálculo de la tasa de respuesta como del ROI dependen de la correcta calibración del modelo y de que las probabilidades de compra no estén sobreestimadas. 

En este caso el modelo está bien ajustado y las probabilidades son precisas, por lo que el ROI elevado que se ha calculado puede ser completamente realista. Las campañas de marketing altamente focalizadas, como la nuestra, pueden generar ROIs muy altos cuando los costos se mantienen bajos y los beneficios por cliente son elevados.

Sin embargo, para asegurar la fiabilidad de este análisis, es recomendable seguir estos pasos para validar y afinar el cálculo del ROI:

1.	Verificar la calibración del modelo: Asegúrate de que las probabilidades de compra reflejan tasas de respuesta realistas y no están infladas, a partir de la mejora continua del modelo.
2.	Simular diferentes escenarios de tasa de respuesta: Probar con diferentes tasas de respuesta para ver cómo impactan en el ROI.
3.	Comparar con benchmarks históricos: Evaluar tasas de respuesta de campañas anteriores o utilizar tasas estándar para campañas similares en tu sector.
4.	Revisar los costos y beneficios reales: Confirmar que todos los costos y beneficios asociados a la campaña están correctamente estimados, evitando cualquier sobreestimación.

Resumiendo:

- Si las probabilidades de compra son precisas y los clientes seleccionados tienen una alta propensión a comprar, tanto la tasa de respuesta elevada como el ROI alto pueden ser completamente viables.
- Validar estas hipótesis mediante la mejora continua del modelo y simulaciones de diferentes tasas de respuesta y costos reales de la campaña es clave para asegurar confianza en los resultados.
